Remaining TODO:

1. Determine correct `zoom` values for each state
2. Run GIFmaker for all

In [ ]:
%matplotlib inline

import numpy as np
import pandas as pd
import config
import imageio

import plotly
import plotly.plotly as py
import plotly.graph_objs as go

from PIL import Image, ImageFont, ImageDraw

In [ ]:
mapbox_access_token = config.mapbox_access_token
plotly.tools.set_credentials_file(username=config.plotly_username, api_key=config.plotly_api_key)

In [ ]:
ge = pd.read_csv("data/election-data-2004-2018.csv")

Same changes as in `explorer.ipynb`.

In [ ]:
ge.at[849, "winlose"] = 0
ge["coalition"].fillna("Unaffiliated", inplace=True)

In [ ]:
years = ge["year"].unique()
coals = ge["coalition"].unique()
states = ge["state"].unique()

In [ ]:
coal_colors = dict(
    BN="rgba(9, 39, 129, 0.5)", 
    Pakatan="rgba(237, 28, 36, 0.5)", 
    PAS="rgba(0, 136, 0, 0.5)",
    Unaffiliated="rgba(26, 26, 26, 0.5)"
)

In [ ]:
def map_data_layout(year, area):    
    lat = geo[area]["lat"]
    lon = geo[area]["lon"]
    key = "malaysia" if (area is "west_malaysia") else str(area)
    zoom = geo[area]["zoom"]
    
    data = go.Data([go.Scattermapbox(lat=lat, lon=lon, mode="markers")])
    layout = go.Layout(
        height=1100, width=1100, autosize=False, hovermode=False,
        margin=go.Margin(l=0, r=0, b=0, t=0, pad=0),
        mapbox=dict(
            layers=[{
                "source": f"https://raw.githubusercontent.com/theianchan/ge14-map/master/data/ge-{str(year)}-{coal.lower()}-{key}.geojson",
                "sourcetype": "geojson",
                "type": "fill",
                "color": coal_colors[coal]
            } for coal in coals],
            accesstoken=mapbox_access_token,
            bearing=0,
            center=dict(lat=lat, lon=lon),
            zoom=zoom,
            style="light",
            pitch=0
        )
    )
    return data, layout

In [ ]:
def get_map(year, area, display=True, save=True):
    data, layout = map_data_layout(year, area)
    if save:
        fname = f"maps/ge-{str(year)}-{str(area)}.png"
        py.image.save_as(go.Figure(data=data, layout=layout), filename=fname)
        print(f"Saved {fname}")
    if display:
        return dict(data=data, layout=layout)

1. Double check Pahang, Selangor
2. Include Putrajaya in KL
3. Create composite for Sabah + Sarawak + Labuan ???

In [ ]:
geo = dict(
    malaysia=dict(
        lat=4.213794, lon=109.397091, zoom=5.00),
    west_malaysia=dict(
        lat=3.950791, lon=101.992306, zoom=6.00),
    perlis=dict(
        lat=6.486549, lon=100.260023, zoom=10.00),
    kedah=dict(
        lat=5.798265, lon=100.620284, zoom=8.00),
    kelantan = dict(
        lat=5.448893, lon=102.073530, zoom=8.00),
    terengganu = dict(
        lat=4.904895, lon=103.031562, zoom=8.00),
    penang = dict(
        lat=5.352609, lon=100.345081, zoom=10.00),
    perak = dict(
        lat=4.791387, lon=100.927375, zoom=8.00),
    pahang = dict(
        lat=3.615012, lon=102.664301, zoom=7.80),
    selangor = dict(
        lat=3.180396, lon=101.418123, zoom=9.00),
    wp_kuala_lumpur = dict(
        lat=3.130236, lon=101.687618, zoom=11.00),
#     wp_putrajaya = dict(
#         lat=2.923853, lon=101.701562, zoom=12.00),
    negeri_sembilan = dict(
        lat=2.793770, lon=102.230789, zoom=9.00),
    melaka = dict(
        lat=2.243490, lon=102.252917, zoom=10.00),
    johor = dict(
        lat=2.000979, lon=103.329079, zoom=8.00),
#     wp_labuan = dict(
#         lat=5.280000, lon=115.240000, zoom=11.00),
#     sabah = dict(
#         lat=5.805978, lon=117.212464, zoom=7.00),
#     sarawak = dict(
#         lat=2.953408, lon=113.444924, zoom=7.00)
)

### Preview maps

In [ ]:
# py.iplot(get_map(2018, "melaka"), filename="ge-2018-melaka")

### Create maps

In [ ]:
for g in geo:
    for year in years:
        get_map(year, g, display=False)

## Process images and create GIFs

In [ ]:
font_heading = ImageFont.truetype("fonts/Lato-Bold.ttf", 60)
font_paragraph = ImageFont.truetype("fonts/Lato-Regular.ttf", 24)
font_color_dark = (34, 34, 34)

In [ ]:
x_lm = 40
y_tm = 40
y_cp = 40 + 84

y_bn = 904
y_ph = 904 + 42
y_gs = 904 + 42 + 42
y_ot = 904 + 42 + 42 + 42

x_ch = 287
h_ch = 30

c_bn = (9, 39, 129)
c_ph = (237, 28, 36)
c_gs = (0, 136, 0)
c_ot = (26, 26, 26)

In [ ]:
def num_seats_won(year, coalition, area):
    seats = ge[(ge["year"] == year) & (ge["coalition"] == coalition)]
    if ((area is "malaysia") or (area is "west_malaysia")):
        return seats["winlose"].sum()
    return seats[seats["state"].apply(lambda x: x.lower().replace(" ", "_")) == area]["winlose"].sum()

In [ ]:
years

In [ ]:
elections = {
    2004: "GE11",
    2008: "GE12",
    2013: "GE13",
    2018: "GE14"
}

In [ ]:
for g in geo:
    for year in years:
        img = Image.open(f"maps/ge-{str(year)}-{str(g)}.png")
        draw = ImageDraw.Draw(img)
        draw.text((x_lm, y_tm), f"{elections[year]} ({g.title().replace('_', ' ').replace('Wp', 'WP')})", font_color_dark, font_heading)
        draw.text((x_lm, y_cp), "Created for datatarik.com", font_color_dark, font_paragraph)

        seats = num_seats_won(year, "BN", g)
        draw.text((x_lm, y_bn), "Barisan National", font_color_dark, font_paragraph)
        draw.rectangle([(x_ch, y_bn), (x_ch + seats * 3, y_bn + h_ch)], fill=c_bn)
        draw.text((x_ch + seats * 3 + 12, y_bn), str(seats), font_color_dark, font_paragraph)

        seats = num_seats_won(year, "Pakatan", g)
        draw.text((x_lm, y_ph), "Pakatan Harapan", font_color_dark, font_paragraph)
        draw.rectangle([(x_ch, y_ph), (x_ch + seats * 3, y_ph + h_ch)], fill=c_ph)
        draw.text((x_ch + seats * 3 + 12, y_ph), str(seats), font_color_dark, font_paragraph)

        seats = num_seats_won(year, "PAS", g)
        draw.text((x_lm, y_gs), "Gagasan Sejahtera", font_color_dark, font_paragraph)
        draw.rectangle([(x_ch, y_gs), (x_ch + seats * 3, y_gs + h_ch)], fill=c_gs)
        draw.text((x_ch + seats * 3 + 12, y_gs), str(seats), font_color_dark, font_paragraph)

        seats = num_seats_won(year, "Unaffiliated", g)
        draw.text((x_lm, y_ot), "Unaffiliated (Other)", font_color_dark, font_paragraph)
        draw.rectangle([(x_ch, y_ot), (x_ch + seats * 3, y_ot + h_ch)], fill=c_ot)
        draw.text((x_ch + seats * 3 + 12, y_ot), str(seats), font_color_dark, font_paragraph)

        fname = f"frames/ge-{str(year)}-{g}.png"
        img.save(fname)
        print(f"Saved {fname}")

In [ ]:
for g in geo:
    frames = []
    for year in years:
        frames.append(imageio.imread(f"frames/ge-{str(year)}-{str(g)}.png"))
    fname = f"gifs/ge-{str(g)}.gif"
    imageio.mimwrite(fname, frames, "GIF", duration=2)
    print(f"Saved {fname}")